# First look

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# Load dataset
df = pd.read_excel(r"D:\Nam_3_2\BME\BME3\data_4_paper.xlsx")
df.info()

In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
# Rename for personal
df.rename(columns={
    'drugsID':'drug_id',
    'order_numbers': 'amount',
    'city': 'district'
}, inplace=True)
df.columns

In [ ]:
# convert the date column to datetime and set it as the index
df.set_index('time_step', inplace=True)
df.index = pd.to_datetime(df.index) #convert the index to a datetime object
df.head()

In [ ]:
df.describe()

## Why log transformation

To reduce the vast number of few numbers in or to reduce
over-fitting, data were processed before model training using log transformation methods

[[1] Andrew Gelman, “You should (usually) log transform your positive data,” Statistical Modeling, Causal Inference, and Social Science, August 21, 2019, 1.](https://statmodeling.stat.columbia.edu/2019/08/21/you-should-usually-log-transform-your-positive-data/)

In [ ]:
print('Before log')
inv_log =  df['amount'].apply(np.exp)
inv_log.hist(bins = 20)

In [ ]:
print('After log')
df['amount'].hist(bins=50)

In [ ]:
# time feature
buffer = df.index
buffer = pd.Series(buffer) # Convert to Series

df['month'] = list(buffer.dt.month) # extract the month
df['year'] = list(buffer.dt.year) # extract the year
df['season'] = list(buffer.dt.month % 4 + 1) # extract the season

df.head()

## Encoding

In [ ]:
print(df['drug_id'].nunique())
print(df['district'].nunique())

In [ ]:
# one-hot for district column
df = pd.get_dummies(df, columns=['district'], drop_first=False) 

# integer encoding for drug_id
# convert the 'drug_id' column to a categorical type
df['drug_id'] = pd.Categorical(df['drug_id'])

# get the integer codes for each category
df['drug_id_idx'] = df['drug_id'].cat.codes

# drop the original column
df = df.drop('drug_id', axis=1)
df.head()

# Test-train spliting

In [ ]:
from sklearn.model_selection import train_test_split

# separate features (X) and target (y)
X = df.drop('amount', axis=1)
y = df['amount']
print(X.shape, y.shape)

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
print(X_train.shape)
print(X_test.shape)

# Build model

In [ ]:
# Plotting function (y_test vs predict)
def plot_line(y_test, predictions):
    plt.figure(figsize=(8, 6))
    plt.scatter(y_test, predictions, c='blue', label='y-test data points')
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', linewidth=2, label='Ideal line')
    plt.xlabel('y_test [unit post-log]')
    plt.ylabel('y_predict [unit post-log]')
    plt.title('Actual vs. Predicted Values')
    plt.legend()
    plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error


# Func compute metrics: r-2, RMSE
# add MAE, MAPE, MSE
def print_score(y_test, y_predict):
    # Compute R-Squared
    r2 = r2_score(y_test, y_predict)
    print(f'R-squared = {r2}')

    # Compute Adjusted R-Squared
    n = len(y_test)  # Number of data points
    p = 1  # Number of predictors (independent variables)
    adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))
    print("Adjusted R-Squared:", adjusted_r2)

    # Compute RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))
    print(f"RMSE: {rmse:.3f}")
    
    # MAE
    mae = mean_absolute_error(y_test, y_predict) 
    print(f"MAE: {mae:.2f}")
    
    # MAPE
#     mape = mean_absolute_percentage_error(y_test, y_predict)
#     print(f"MAPE: {mape}")

    # MSE
    mse = mean_squared_error(y_test, y_predict) 
    print(f"MSE: {mse:.2f}")

# LGBM

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

# create an instance of HistGradientBoostingRegressor
model = HistGradientBoostingRegressor()

# fit the model to training data
model.fit(X_train, y_train)

# evaluate
print("Train")
train_pred = model.predict(X_train)
print_score(y_train, train_pred)

print("Test")
test_pred = model.predict(X_test)
print_score(y_test, test_pred)
plot_line(y_test, test_pred)

In [ ]:
model.get_params()

# Fine tune

In [ ]:
# Import the necessary modules
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter grid to search over
random_grid = {
    'loss': ['squared_error', 'absolute_error', 'gamma', 'poisson', 'quantile'],
    'learning_rate': [0.1, 0.03, 0.003],
    'max_iter': [50, 100, 200, 500],
    'max_leaf_nodes': [7, 14, 21, 28, 31, 50],
    'max_depth': [-1, 3, 5],
    'min_samples_leaf': [1, 2, 4, 10, 20],
    'l2_regularization': [0.0, 0.1, 0.5, 1.0],
}

# Create the base model to tune
hgbr = HistGradientBoostingRegressor(random_state=42)

# Create the random search object with 5-fold cross validation and 100 iterations
hgbr_random = RandomizedSearchCV(
    estimator=hgbr,
    param_distributions=random_grid,
    n_iter=100,
    cv=5,
    scoring='r2',
    verbose=10,
    random_state=42,
    n_jobs=-1
)

# Fit the random search model on the data
hgbr_random.fit(X, y)

# Print the best parameters and score
print(hgbr_random.best_params_)
print(hgbr_random.best_score_)


In [ ]:
# # Print the best parameters and score
# print(model.best_params_)
# print(model.best_score_)

# evaluate
print("Train")
train_pred = hgbr_random.predict(X_train)
print_score(y_train, train_pred)

print("Test")
test_pred = hgbr_random.predict(X_test)
print_score(y_test, test_pred)
plot_line(y_test, test_pred)